# Text Summarization using BART Transformer

In [ ]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
# LOADING THE DATASET
from datasets import load_dataset

ds = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [ ]:
ds['train'][1]['dialogue']

"#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little."

In [ ]:
ds['train'][1]['summary']

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

### WITHOUT FINE - TUNING

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
article_1 = ds['train'][1]['dialogue']

In [ ]:
pipe(article_1, max_length=20, min_length=10, do_sample=False)

[{'summary_text': 'Ricky has received his Polio, Tetanus and Hepatitis B shots.'}]

In [ ]:
ds['train'][1]['summary']

'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.'

### WITH FINE - TUNING

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [ ]:
# tokenization
def preprocess_function(batch):
  source = batch['dialogue']
  target = batch['summary']
  source_ids = tokenizer(source, padding='max_length', truncation=True, max_length=128)
  target_ids = tokenizer(source, padding='max_length', truncation=True, max_length=128)

  labels = target_ids['input_ids']
  labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels]

  return{
          'input_ids': source_ids['input_ids'],
          'attention_mask': source_ids['attention_mask'],
          'labels': labels
      }

In [ ]:
df_source = ds.map(preprocess_function, batched=True, batch_size=1000, num_proc=4)

Map (num_proc=4):   0%|          | 0/12460 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
# training arguments
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='/content',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,
    remove_unused_columns=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_source['train'],
    eval_dataset=df_source['test']
)


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: osenisamuel698 (osenisamuel698-lincoln-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.011000
1000,0.003600
1500,0.002700
2000,0.001500
2500,0.000800
3000,0.000500


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3116, training_loss=0.003221724620531283, metrics={'train_runtime': 3413.243, 'train_samples_per_second': 7.301, 'train_steps_per_second': 0.913, 'total_flos': 6750530835578880.0, 'train_loss': 0.003221724620531283, 'epoch': 2.0})

In [ ]:
eval_results = trainer.evaluate()

In [ ]:
eval_results

{'eval_loss': 0.015150470659136772,
 'eval_runtime': 47.6185,
 'eval_samples_per_second': 31.5,
 'eval_steps_per_second': 3.948,
 'epoch': 2.0}

## SAVING THE MODEL

In [ ]:
model.save_pretrained('/content/model_directory')
tokenizer.save_pretrained('/content/model_directory')

('/content/model_directory/tokenizer_config.json',
 '/content/model_directory/special_tokens_map.json',
 '/content/model_directory/vocab.json',
 '/content/model_directory/merges.txt',
 '/content/model_directory/added_tokens.json',
 '/content/model_directory/tokenizer.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/model_directory')
model = AutoModelForSeq2SeqLM.from_pretrained('/content/model_directory')

def summarize(blog_post):
  #Tokenize the input blog post
  inputs = tokenizer(blog_post, max_length = 1024, truncation = True, return_tensors = 'pt' )

  #Generate the summary
  summary_ids = model.generate(inputs['input_ids'], max_length = 150, num_beams = 4,  no_repeat_ngram_size=2, min_length = 40, early_stopping = True)

  #Decode the summary
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens= True)

  return summary

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [ ]:
blog_post = """
Presidents aren’t supposed to direct IRS investigations
US law specifically prohibits presidents from directing the IRS to investigate anyone in a section entitled: “Prohibition on executive branch influence over taxpayer audits and other investigations.”

While the IRS falls under the Treasury Department, it’s important that it be as protected from politics as possible. That’s why the IRS has only two politically appointed officials, according to Mark Mazur, who was assistant secretary of treasury for tax policy at the outset of the Biden administration

The US has higher voluntary tax payment rates than other countries, Mazur told me, “because people feel that their interactions with the tax system are fair and based on law.”

If the IRS is suddenly used for political purposes, that trust could be destroyed. During the Obama administration, for instance, the IRS became embroiled in a bona fide scandal when a Treasury Department investigation found the IRS delayed conferring tax-exempt status on conservative groups.

If the IRS did find that its tax-exempt status should be revoked, Harvard would need to be warned and given an opportunity to contest the finding. It would also have the opportunity to challenge the IRS in court.

There is already a lot of chaos at the IRS under the new Trump administration. Multiple acting commissioners have resigned, apparently the result a standoff over whether tax data could be used by immigration officials.

It would not be unprecedented for a university to lose its tax-exempt status
Back in 1983, the Supreme Court agreed that Bob Jones University should not be tax-exempt because, at the time, it banned interracial relationships among its students.

The university didn’t drop its interracial marriage policy until 2000 — in an announcement on CNN’s Larry King Live, coincidentally — although it did not regain its tax-exempt status until 2017.

The US has now come full circle to the point that one of the main gripes Trump has with Harvard is its diversity programs.
"""
summary = summarize(blog_post)
print(f'Summary : {summary}')

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1666: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Summary : #Presidents aren’n't supposed to direct IRS investigations
US law specifically prohibits presidents from directing the IRS to investigate anyone in a section entitled: “Prohibition on executive branch influence over taxpayer audits and other investigations.”


